# import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 63.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitli

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import ast

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn import preprocessing
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModel
from torchvision.transforms import v2  # torchvision.transforms.v2 에서 CutMix 사용

import warnings
warnings.filterwarnings(action='ignore')


# Randomseed 고정

In [ ]:
CFG = {
    'EPOCHS': 100,
    'IMG_SIZE': 224,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

def set_seed(seed=CFG['SEED']):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    pl.seed_everything(seed)
set_seed()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

INFO:lightning_fabric.utilities.seed:Seed set to 41


# Data load & Preprocessing

In [ ]:
'''

# img path 수정
folder_path = '/content/drive/MyDrive/basic_DL/data'
train = pd.read_csv(f'{folder_path}/train.csv')
#test = pd.read_csv(f'{folder_path}/test.csv')

train['img_path'] = train['img_path'].apply(lambda x: folder_path + x[1:])
train['upscale_img_path'] = train['upscale_img_path'].apply(lambda x: folder_path + x[1:])

'''

In [ ]:
'''

# train-validation split
train_df, val_df = train_test_split(train, test_size=0.3, stratify=train['label'], random_state=CFG['SEED'])

# Label Encoding
le = preprocessing.LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
val_df['label'] = le.transform(val_df['label'])

'''

In [ ]:
'''

# 업스케일링 데이터 추가하여 train_df 확장
train_expanded_df = pd.concat([
    train_df,  # 원본
    train_df.assign(img_path=train_df['upscale_img_path'])  # 업스케일링
], ignore_index=True)

print("원본+업스케일링이 추가된 train_expanded_df:", len(train_expanded_df))

'''

원본+업스케일링이 추가된 train_expanded_df: 22166


In [ ]:
def scale_handler(x, scale_factor=0.25):
    if isinstance(x, str):
        x = ast.literal_eval(x)

    if x != 0:
        scaled_bbox = [int(coord * scale_factor) for coord in x]
        return scaled_bbox

    else:
        return 0

In [ ]:
# train df
master_df = pd.read_csv("/content/drive/MyDrive/2025-1 KUBIG/finetuning/master_df.csv", index_col=0)

folder_path = '/content/drive/MyDrive/2025-1 KUBIG/dataset/train/' # 경로 변경

master_df['img_path'] = master_df['img_filename'].apply(lambda x: folder_path + x + ".jpg")
master_df['merge_box'] = master_df['merge_box'].apply(lambda x : scale_handler(x))

master_df = master_df[master_df["merge_box"] !=0]
master_df = master_df[master_df["img_filename"] != "TRAIN_04810"]
# box 영역이 너무 작아 이미지 처리 불가능해진 데이터 제외

master_df.reset_index(inplace=True, drop=True)

master_df.head()

,img_path,upscale_img_path,label,img_filename,yolo_box,yolo_conf,unet_box,unet_conf,merge_box
0,/content/drive/MyDrive/2025-1 KUBIG/dataset/tr...,./upscale_train/TRAIN_00000.png,Ruddy Shelduck,TRAIN_00000,0,0.000000,"[3, 64, 188, 179]",0.985534,"[0, 16, 47, 44]"
1,/content/drive/MyDrive/2025-1 KUBIG/dataset/tr...,./upscale_train/TRAIN_00001.png,Gray Wagtail,TRAIN_00001,0,0.000000,"[93, 79, 147, 234]",0.951921,"[23, 19, 36, 58]"
2,/content/drive/MyDrive/2025-1 KUBIG/dataset/tr...,./upscale_train/TRAIN_00002.png,Indian Peacock,TRAIN_00002,0,0.000000,"[81, 53, 227, 239]",0.895013,"[20, 13, 56, 59]"
3,/content/drive/MyDrive/2025-1 KUBIG/dataset/tr...,./upscale_train/TRAIN_00003.png,Common Kingfisher,TRAIN_00003,"[2, 93, 92, 236]",0.255118,"[0, 61, 126, 244]",0.980908,"[0, 15, 31, 61]"
4,/content/drive/MyDrive/2025-1 KUBIG/dataset/tr...,./upscale_train/TRAIN_00004.png,Common Kingfisher,TRAIN_00004,"[0, 57, 106, 175]",0.487455,"[0, 37, 152, 190]",0.963232,"[0, 9, 38, 47]"


In [ ]:
# train-validation split
train_df, val_df = train_test_split(master_df, test_size=0.3, stratify=master_df['label'], random_state=CFG['SEED'])

# Label Encoding
le = preprocessing.LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
val_df['label'] = le.transform(val_df['label'])

# CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, transforms, processor):
        self.df = df
        self.transforms = transforms
        self.processor = processor

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_path = row['img_path']
        label = row['label']

        image = cv2.imread(img_path)
        # 만약 이미지가 None이면 경고 메시지 출력 후 다음 샘플로 넘어감
        if image is None:
            print(f"Warning: 이미지 로드 실패 - {img_path}. 다른 샘플로 넘어갑니다.")
            new_index = (index + 1) % len(self)
            return self.__getitem__(new_index)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # --- 동헌 추가 부분 ~ #

        bbox = row['merge_box']

        if isinstance(bbox, str):
            bbox = ast.literal_eval(bbox)

        if (bbox != 0) and (bbox != "0"):
            x_min, y_min, x_max, y_max = bbox
            image = image[y_min:y_max, x_min:x_max]

        # ~ 동헌 추가 부분 --- #

        image = self.transforms(image=image)['image'] #augmentation

        inputs = self.processor(image, return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze(0)  # (1, C, H, W) -> (C, H, W)

        return {
            "pixel_values": pixel_values,
            "labels": torch.tensor(int(label), dtype=torch.long)
        }

    def __len__(self):
        return len(self.df)

# Augmentation

In [ ]:
train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    ToTensorV2()
])

# Cutmix

In [ ]:
num_classes = len(le.classes_)

# CutMix 객체 생성 (num_classes 적용)
cutmix = v2.CutMix(num_classes=num_classes)

In [ ]:
# CutMix 사용 여부 설정 (True: 사용, False: 사용 안 함)
USE_CUTMIX = True

# cutmix 적용을 위한 collate_fn (학습용)
def train_collate_fn(batch):
    images = [item["pixel_values"] for item in batch]
    labels = [item["labels"] for item in batch]
    images = torch.stack(images)
    labels = torch.tensor(labels, dtype=torch.long)
    # 학습 시에만 CutMix 적용 (USE_CUTMIX가 True일 경우)
    if USE_CUTMIX and cutmix is not None:
        images, labels = cutmix(images, labels)
    return {"pixel_values": images, "labels": labels}

# 검증용 collate_fn (CutMix 미적용)
def val_collate_fn(batch):
    images = [item["pixel_values"] for item in batch]
    labels = [item["labels"] for item in batch]
    images = torch.stack(images)
    labels = torch.tensor(labels, dtype=torch.long)
    return {"pixel_values": images, "labels": labels}

# Model-BEit2

In [ ]:
model_name = "microsoft/beit-base-patch16-224-pt22k-ft22k"
processor = AutoImageProcessor.from_pretrained(model_name, do_normalize=False)

preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# 데이터셋 생성
train_dataset = CustomDataset(train_df, train_transform, processor)
val_dataset = CustomDataset(val_df, train_transform, processor)

train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=4, collate_fn=train_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4, collate_fn=val_collate_fn)

In [ ]:
class BEiTClassifier(pl.LightningModule):
    def __init__(self, num_classes, model_name=model_name, learning_rate=CFG['LEARNING_RATE']):
        super().__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.backbone = AutoModel.from_pretrained(model_name)
        latent_dim = self.backbone.config.hidden_size
        self.classifier = nn.Linear(latent_dim, num_classes)

    def forward(self, pixel_values):
        outputs = self.backbone(pixel_values=pixel_values)
        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            pooled_output = outputs.pooler_output
        else:
            pooled_output = outputs.last_hidden_state[:, 0]
        logits = self.classifier(pooled_output)
        return logits

    def training_step(self, batch, batch_idx):
        pixel_values = batch["pixel_values"]
        labels = batch["labels"]
        logits = self.forward(pixel_values)
        loss = F.cross_entropy(logits, labels)
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        pixel_values = batch["pixel_values"]
        labels = batch["labels"]
        logits = self.forward(pixel_values)
        loss = F.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        # 여기서 개별 배치 f1는 로그에 남기되, epoch 단위에서 재계산하도록 할 수 있음
        batch_f1 = f1_score(labels.detach().cpu().numpy(), preds.detach().cpu().numpy(), average='macro')
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)
        self.log("val_f1", batch_f1, prog_bar=True, on_epoch=True)
        # 배치의 예측과 정답을 반환해 validation_epoch_end에서 전체 F1 계산 가능
        return {"loss": loss, "preds": preds, "labels": labels}

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG['EPOCHS'])
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

# ModelCheckpoint 콜백 정의 (val_f1 지표가 최대일 때를 기준으로)
checkpoint_callback = ModelCheckpoint(
    monitor="val_f1",           # 검증 단계에서 기록한 f1 지표를 모니터링
    mode="max",
    save_top_k=1,               # 최고의 모델 1개만 저장
    verbose=True,
    dirpath="/content/drive/MyDrive/basic_DL/results",  # 체크포인트 저장 폴더
    filename="best-checkpoint" # 저장될 체크포인트 파일 이름
)

In [ ]:
# 조기종료
from pytorch_lightning.callbacks import EarlyStopping

early_stop_callback = EarlyStopping(
    monitor="val_loss",     # 모니터링할 지표
    min_delta=0.00,         # 개선으로 간주될 최소 변화량
    patience=5,             # 5에폭 동안 개선이 없으면 종료
    verbose=True,
    mode="min"
)

# Train

In [ ]:
model = BEiTClassifier(num_classes=num_classes, learning_rate=CFG['LEARNING_RATE'])

# Trainer 생성 (accelerator="auto"로 GPU 사용 가능 시 자동 선택)
trainer = pl.Trainer(
    max_epochs=CFG['EPOCHS'],
    accelerator="auto",
    devices=1,
    precision=16,
    callbacks=[checkpoint_callback, early_stop_callback]
)

trainer.fit(model, train_loader, val_loader)

config.json:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/414M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


model.safetensors:   0%|          | 0.00/414M [00:00<?, ?B/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params | Mode 
-------------------------------------------------
0 | backbone   | BeitModel | 85.8 M | eval 
1 | classifier | Linear    | 19.2 K | train
-------------------------------------------------
85.8 M    Trainable params
0         Non-trainable params
85.8 M    Total params
343.125   Total estimated model params size (MB)
1         Modules in train mode
250       Modules in eval mode


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 86, global step 28362: 'val_f1' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 87, global step 28688: 'val_f1' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_loss did not improve in the last 5 records. Best score: 0.596. Signaling Trainer to stop.
INFO:pytorch_lightning.utilities.rank_zero:Epoch 88, global step 29014: 'val_f1' was not in top 1


# Inference

In [ ]:
test_transforms = A.Compose([
    A.Resize(256, 256),
    ToTensorV2()
])

class TestDataset(Dataset):
    def __init__(self, df, transforms, processor):
        self.df = df
        self.transforms = transforms
        self.processor = processor

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['img_path']
        image = cv2.imread(img_path)
        # 이미지 로드 실패 시 경고 출력 후 None 반환하여 건너뜁니다.
        if image is None:
            print(f"Warning: 이미지 로드 실패 - {img_path}. 건너뜁니다.")
            return None
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transforms:
            image = self.transforms(image=image)['image']
        inputs = self.processor(image, return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze(0)  # (1, C, H, W) -> (C, H, W)
        return {"pixel_values": pixel_values}

    def __len__(self):
        return len(self.df)

def test_collate_fn(batch):
    # None인 항목 제거
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        raise ValueError("모든 이미지 로드에 실패했습니다.")
    images = [item["pixel_values"] for item in batch]
    images = torch.stack(images)
    return {"pixel_values": images}


In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/2025-1 KUBIG/finetuning/test_df_2.csv")
test_df

,Unnamed: 0,id,img_path,finetuned_box,img_name
0,0,TEST_00000,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[24, 16, 44, 34]",TEST_00000.jpg
1,1,TEST_00001,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[39, 17, 55, 51]",TEST_00001.jpg
2,2,TEST_00002,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[26, 4, 49, 39]",TEST_00002.jpg
3,3,TEST_00003,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[27, 23, 52, 59]",TEST_00003.jpg
4,4,TEST_00004,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,0,TEST_00004.jpg
...,...,...,...,...,...
6781,6781,TEST_06781,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[3, 16, 34, 46]",TEST_06781.jpg
6782,6782,TEST_06782,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[15, 17, 63, 55]",TEST_06782.jpg
6783,6783,TEST_06783,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[30, 19, 54, 56]",TEST_06783.jpg
6784,6784,TEST_06784,/content/drive/MyDrive/2025-1 KUBIG/dataset/te...,"[3, 1, 53, 64]",TEST_06784.jpg


In [ ]:
'''

#테스트 데이터 로드 설정
test_csv_path = "/content/drive/MyDrive/basic_DL/data/test.csv"
test_df = pd.read_csv(test_csv_path)
folder_path = "/content/drive/MyDrive/basic_DL/data"
test_df['img_path'] = test_df['img_path'].apply(lambda x: os.path.join(folder_path, x.strip("./")))

'''

In [ ]:
model_name = "microsoft/beit-base-patch16-224-pt22k-ft22k"
processor = AutoImageProcessor.from_pretrained(model_name, do_normalize=False)

test_dataset = TestDataset(test_df, transforms=test_transforms, processor=processor)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4, collate_fn=test_collate_fn)

In [ ]:
best_checkpoint_path = "/content/drive/MyDrive/basic_DL/results/best-checkpoint-v2.ckpt"

model = BEiTClassifier.load_from_checkpoint(best_checkpoint_path)
model.to(device)
model.eval()

In [ ]:
predictions = []
model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        # 배치 딕셔너리에서 pixel_values만 추출하여 device로 이동
        pixel_values = batch["pixel_values"].to(device)  # [B, C, H, W]
        logits = model(pixel_values)  # forward() 호출; logits shape: [B, num_classes]
        preds = torch.argmax(logits, dim=1)  # [B]
        predictions.extend(preds.cpu().numpy())


In [ ]:
# train_csv_path = "/content/drive/MyDrive/basic_DL/data/train.csv"
# train_df = pd.read_csv(train_csv_path)
le = preprocessing.LabelEncoder()
le.fit(master_df["label"])

# 예측 결과(숫자)를 원래 클래스명으로 역변환
final_labels = le.inverse_transform(np.array(predictions))

# sample_submission.csv 파일을 불러와 예측 결과 적용
submission_csv_path = "/content/drive/MyDrive/2025-1 KUBIG/dataset/sample_submission.csv"
submission_df = pd.read_csv(submission_csv_path)
submission_df["label"] = final_labels
submission_df.to_csv("/content/drive/MyDrive/2025-1 KUBIG/finetuning/submission_BEit2.csv", index=False)

from google.colab import files
files.download("submission_BEit2.csv")